In [1]:
import torch

x = torch.rand(2, 15, 128, 128) > 0.5
y = torch.rand(2, 10, 128, 128) > 0.5
x[0, 5, 10:70, 20:100] = True
y[0, 3, 10:70, 20:100] = True

z = (x.unsqueeze(1) == y.unsqueeze(2)).sum(dim=(3, 4)) / (x.unsqueeze(1) | y.unsqueeze(2)).sum(dim=(3, 4))
z.shape

torch.Size([2, 10, 15])

In [3]:
from scipy.optimize import linear_sum_assignment
import torch


def iou_cost(prediction, target):
    return (prediction.unsqueeze(1) == target.unsqueeze(2)).sum(dim=(3, 4)) / (prediction.unsqueeze(1) | target.unsqueeze(2)).sum(dim=(3, 4))


class HungarianMatcher(torch.nn.Module):

    def __init__(self) -> None:
        super().__init__()

    @torch.no_grad()
    def forward(self, prediction, target):
        matches = []
        cost_matrix = iou_cost(prediction, target)
        for batch in range(cost_matrix.shape[0]):
            cm = 1 - cost_matrix[batch]
            rows, cols = linear_sum_assignment(cm.cpu().numpy())
            matches.append(torch.stack([torch.tensor(rows), torch.tensor(cols)], dim=1))
        return matches


In [4]:
HungarianMatcher()(x, y)

[tensor([[ 0,  3],
         [ 1, 10],
         [ 2,  0],
         [ 3,  5],
         [ 4, 14],
         [ 5, 13],
         [ 6, 12],
         [ 7,  8],
         [ 8, 11],
         [ 9,  7]]),
 tensor([[ 0,  0],
         [ 1, 12],
         [ 2,  6],
         [ 3,  2],
         [ 4,  1],
         [ 5,  9],
         [ 6,  7],
         [ 7,  8],
         [ 8,  4],
         [ 9, 14]])]